# Лабораторная работа №4 (проведение исследований с алгоритмом RandomForest)

In [32]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score, max_error


In [33]:
car_dataset = fetch_ucirepo(id=9)

In [34]:
df = pd.DataFrame(data=car_dataset.data.features)
df['mpg'] = car_dataset.data.targets

In [35]:
target_column = "mpg"

In [36]:
df["horsepower"] = df["horsepower"].fillna(df["horsepower"].median())

Разобьем выборку на обучающую и валидационную

In [37]:
X = df.drop(columns=target_column, inplace=False)
y = df[target_column]

In [38]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

## 2. Создание бейзлайна и оценка качества

### Обучение модели

Определим модель и обучим модель

In [39]:
model = RandomForestRegressor()
model.fit(X_train, y_train)

RandomForestRegressor()

Сделаем предсказания

In [40]:
train_predict = model.predict(X_train)
valid_predict = model.predict(X_valid)

### Оценка качества модели

In [41]:
# Расчет метрик для тренировочных предсказаний
train_mae = mean_absolute_error(y_train, train_predict)
train_mape = mean_absolute_percentage_error(y_train, train_predict) * 100  # Преобразуем в проценты
train_r2 = r2_score(y_train, train_predict)
train_max_error = max_error(y_train, train_predict)

# Расчет метрик для валидационных предсказаний
valid_mae = mean_absolute_error(y_valid, valid_predict)
valid_mape = mean_absolute_percentage_error(y_valid, valid_predict) * 100  # Преобразуем в проценты
valid_r2 = r2_score(y_valid, valid_predict)
valid_max_error = max_error(y_valid, valid_predict)

# Сводка метрик в таблицу для удобного отображения
metrics_df = pd.DataFrame({
    'Метрика': ['MAE', 'MAPE (%)', 'R²', 'Max Error'],
    'Тренировочная выборка': [train_mae, train_mape, train_r2, train_max_error],
    'Валидационная выборка': [valid_mae, valid_mape, valid_r2, valid_max_error]
})

In [42]:
metrics_df.head()

,Метрика,Тренировочная выборка,Валидационная выборка
0,MAE,0.753324,1.551937
1,MAPE (%),3.168184,7.054714
2,R²,0.980064,0.917670
3,Max Error,7.993000,6.953000


## 3. Улучшение бейзлайна

### Препроцессинг данных

In [43]:
df_optimized = df.copy()

In [44]:
# Обработка выбросов
def remove_outliers(data, columns, threshold=1.5):
    for col in columns:
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - threshold * IQR
        upper_bound = Q3 + threshold * IQR
        data = data[(data[col] >= lower_bound) & (data[col] <= upper_bound)]
    return data

df_optimized = remove_outliers(df_optimized, df_optimized.columns)

In [45]:
X = df_optimized.drop(target_column, axis=1)
y = df_optimized[target_column]

Скалирование данных

In [46]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [47]:
new_X_train, new_X_valid, new_y_train, new_y_valid = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

### Обучение модели с новыми данными

Переопределим модель

In [48]:
model = RandomForestRegressor()

In [49]:
model.fit(new_X_train, new_y_train)

RandomForestRegressor()

Сделаем предсказания

In [50]:
train_predict = model.predict(new_X_train)
valid_predict = model.predict(new_X_valid)

### Оценка качетсва модели с улучшеным бейзлайном

In [51]:
# Расчет метрик для тренировочных предсказаний
train_mae = mean_absolute_error(new_y_train, train_predict)
train_mape = mean_absolute_percentage_error(new_y_train, train_predict) * 100  # Преобразуем в проценты
train_r2 = r2_score(new_y_train, train_predict)
train_max_error = max_error(new_y_train, train_predict)

# Расчет метрик для валидационных предсказаний
valid_mae = mean_absolute_error(new_y_valid, valid_predict)
valid_mape = mean_absolute_percentage_error(new_y_valid, valid_predict) * 100  # Преобразуем в проценты
valid_r2 = r2_score(new_y_valid, valid_predict)
valid_max_error = max_error(new_y_valid, valid_predict)

# Сводка метрик в таблицу для удобного отображения
metrics_df = pd.DataFrame({
    'Метрика': ['MAE', 'MAPE (%)', 'R²', 'Max Error'],
    'Тренировочная выборка': [train_mae, train_mape, train_r2, train_max_error],
    'Валидационная выборка': [valid_mae, valid_mape, valid_r2, valid_max_error]
})

In [52]:
metrics_df.head()

,Метрика,Тренировочная выборка,Валидационная выборка
0,MAE,0.706395,1.856974
1,MAPE (%),3.019418,8.035943
2,R²,0.981692,0.862392
3,Max Error,6.774000,10.692000


### Резюме:
На качество модели повлияли, как входные данные, так и параметры самой модели:
- При помощи квантилей я удалил выбросы
- Скалировал данные StandartScaller'ом
- Избавился от высокой кореляции фичей

## 4. Имплементация алгоритма машинного обучения

### Реализация модели

Импортируем модель

In [53]:
from RForest import MyRandomForestRegressor

### Обучение 

In [54]:
model = MyRandomForestRegressor()
model.fit(X_train, y_train)

In [55]:
train_predict = model.predict(X_train)
valid_predict = model.predict(X_valid)

###  Оценка качества модели

In [56]:
# Расчет метрик для тренировочных предсказаний
train_mae = mean_absolute_error(y_train, train_predict)
train_mape = mean_absolute_percentage_error(y_train, train_predict) * 100  # Преобразуем в проценты
train_r2 = r2_score(y_train, train_predict)
train_max_error = max_error(y_train, train_predict)

# Расчет метрик для валидационных предсказаний
valid_mae = mean_absolute_error(y_valid, valid_predict)
valid_mape = mean_absolute_percentage_error(y_valid, valid_predict) * 100  # Преобразуем в проценты
valid_r2 = r2_score(y_valid, valid_predict)
valid_max_error = max_error(y_valid, valid_predict)

# Сводка метрик в таблицу для удобного отображения
metrics_df = pd.DataFrame({
    'Метрика': ['MAE', 'MAPE (%)', 'R²', 'Max Error'],
    'Тренировочная выборка': [train_mae, train_mape, train_r2, train_max_error],
    'Валидационная выборка': [valid_mae, valid_mape, valid_r2, valid_max_error]
})

In [57]:
metrics_df.head()

,Метрика,Тренировочная выборка,Валидационная выборка
0,MAE,1.564837,1.742037
1,MAPE (%),6.632767,7.839791
2,R²,0.928274,0.903476
3,Max Error,9.057291,8.044608


### Обучение с улучшенным бейзлайном

In [58]:
model = Pipeline((
    ("pca", PCA(n_components=4)),
    ("model", MyRandomForestRegressor())
))

In [59]:
model.fit(new_X_train, new_y_train)

Pipeline(steps=[('pca', PCA(n_components=4)),
                ('model',
                 <RForest.MyRandomForestRegressor object at 0x000002A8F3B97AF0>)])

In [60]:
train_predict = model.predict(new_X_train)
valid_predict = model.predict(new_X_valid)

###  Оценка качества модели

In [61]:
# Расчет метрик для тренировочных предсказаний
train_mae = mean_absolute_error(new_y_train, train_predict)
train_mape = mean_absolute_percentage_error(new_y_train, train_predict) * 100  # Преобразуем в проценты
train_r2 = r2_score(new_y_train, train_predict)
train_max_error = max_error(new_y_train, train_predict)

# Расчет метрик для валидационных предсказаний
valid_mae = mean_absolute_error(new_y_valid, valid_predict)
valid_mape = mean_absolute_percentage_error(new_y_valid, valid_predict) * 100  # Преобразуем в проценты
valid_r2 = r2_score(new_y_valid, valid_predict)
valid_max_error = max_error(new_y_valid, valid_predict)

# Сводка метрик в таблицу для удобного отображения
metrics_df = pd.DataFrame({
    'Метрика': ['MAE', 'MAPE (%)', 'R²', 'Max Error'],
    'Тренировочная выборка': [train_mae, train_mape, train_r2, train_max_error],
    'Валидационная выборка': [valid_mae, valid_mape, valid_r2, valid_max_error]
})

In [62]:
metrics_df.head()

,Метрика,Тренировочная выборка,Валидационная выборка
0,MAE,1.618669,2.269465
1,MAPE (%),6.969521,9.756745
2,R²,0.917640,0.813318
3,Max Error,6.991412,15.617968
